In [1]:
import os

import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers
from keras.models import load_model
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import auc
import random

In [2]:
def get_col1(fn):
    finame = r'E:/NingLab/object/khib/transfer/experiment/' + str.upper(fn) + r'/A0A0G2JSG6_28.' + str.lower(fn)
    with open(finame) as file:
        values = file.read().rstrip().split('\t')
        i = 0
        columns = []
        while i < len(values):
            columns.append(str(i))
            i += 1
    return columns, len(values)

In [3]:
def prep_dataset1(feature_name):
    global dataset
    col, size = get_col1(feature_name)
    x = dataset[col]
    ID = dataset['pepname']
    return x, ID, size

In [4]:
feature_list = ['OBC']
for feature in feature_list:
    for count in range(1, 6):  # 从1到5遍历count
        model = load_model(f'E:/NingLab/object/khib/transfer/models/DNN/%s/%s_DNN_%d.h5' % (feature, feature, count))
        dataset = pd.read_csv(f'E:/NingLab/object/khib/transfer/experiment/%s_dataset.csv' % feature)
        data, peplist, data_size = prep_dataset1(feature)
        print(feature)
        y_score = model.predict(data)
        df_y = pd.concat([pd.DataFrame(list(peplist), columns=['pepname']),
                          pd.DataFrame(list(y_score), columns=['score'])], axis=1)
        path = f'E:/NingLab/object/khib/transfer/DNN_result_for_experiment/'
        if not os.path.exists(path):
            os.makedirs(path)
        df_y.to_csv(f'{path}{feature}_scores_{count}.csv', index=False)
        #df_y.to_csv(path + f'%s_scores_%.csv' % (feature,count), index=False)

OBC
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step


OBC
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 


OBC
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 


OBC
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step


OBC
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step


In [5]:
import pandas as pd

# 存储'A0A0G2JSH2'的得分和排名
scores_and_ranks = []

for count in range(1, 6):
    # 加载CSV文件
    df = pd.read_csv(f'E:/NingLab/object/khib/transfer/DNN_result_for_experiment/OBC_scores_{count}.csv')
    
    # 筛选出'A0A0G2JSH2'的得分
    score = df[df['pepname'] == 'A0A0G2JSH2_74']['score'].values[0]
    
    # 计算排名
    ranks = df['score'].rank(ascending=False).astype(int)
    rank_of_A0A0G2JSH2 = ranks[df[df['pepname'] == 'A0A0G2JSH2_74'].index[0]]
    
    # 将得分和排名加入到列表中
    scores_and_ranks.append((score, rank_of_A0A0G2JSH2))

# 打印结果
for count, (score, rank) in enumerate(scores_and_ranks, start=1):
    print(f"DNN{count}中'A0A0G2JSH2'的得分为：{score}，排名为：{rank}")


DNN1中'A0A0G2JSH2'的得分为：0.82365346，排名为：221
DNN2中'A0A0G2JSH2'的得分为：0.84729797，排名为：247
DNN3中'A0A0G2JSH2'的得分为：0.89957815，排名为：132
DNN4中'A0A0G2JSH2'的得分为：0.6336696，排名为：538
DNN5中'A0A0G2JSH2'的得分为：0.79046834，排名为：399


In [6]:
# 0.90 132

In [7]:
def auc_11(fileplace):
    features = ['ACF', 'ASA', 'AAINDEX', 'BTA', 'CKSAAP', 'GPS', 'OBC', 'PSEAAC', 'PSSM', 'SS', 'transformer']
    scorelist = []
    
    # 读取第一个文件以获得pepname
    first_file = fileplace + r'/' + features[0] + r'.csv'
    df_first = pd.read_csv(first_file)
    peptidenames = df_first['pepname']
    
    # 初始化一个 DataFrame 来保存所有的分数
    all_scores = pd.DataFrame(peptidenames, columns=['pepname'])
    
    # 处理每个文件

    for feature in features:
        df = pd.read_csv(fileplace + r'/' + feature + r'.csv')
        # 根据 pepname 进行匹配和添加分数
        df = df[df['pepname'].isin(peptidenames)]
        df = df[['pepname', 'score']]
        all_scores = pd.merge(all_scores, df, on='pepname', how='left', suffixes=('', f'_{feature}'))
    
    # 将分数列提取出来
    score_columns = [col for col in all_scores.columns if col != 'pepname']
    
    # 提取所有的分数列
    scorelist = all_scores[score_columns].values
    
    return np.array(scorelist)

In [8]:
def get_name_and_label():
    df = pd.read_csv('E:/NingLab/object/khib/transfer/DNN_result_for_experiment/score/ASA.csv')
    name = df['pepname']
    name = name.values
    return name

In [13]:
for count in range(1, 6):
    data = auc_11('E:/NingLab/object/khib/transfer/DNN_result_for_experiment/score')
    iDNN = load_model(f'E:/NingLab/object/khib/transfer/models/integrated_DNN/iDNN_%d.h5' % count)
    #iDNN = load_model(f'E:/QD065LPSc/Ksuc/models/integrated_DNN/iDNN_%d.h5' % count)
    score = iDNN.predict(data)
    peplist = get_name_and_label()
    df_y = pd.concat([pd.DataFrame(list(peplist), columns=['pepname']),
                  pd.DataFrame(list(score), columns=['score'])], axis=1)
    df_y.to_csv(f'E:/NingLab/object/khib/transfer/models/iDNN_%d_result_for_experiment.csv' % count, index=False)

19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 


19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 


19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 


19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 


19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 


In [14]:
# 存储'A0A0G2JSH2'的得分和排名
scores_and_ranks = []

for count in range(1, 6):
    # 加载CSV文件
   
    df = pd.read_csv(f'E:/NingLab/object/khib/transfer/models/iDNN_%d_result_for_experiment.csv' % count)
    
    # 筛选出'A0A0G2JSH2'的得分
    score = df[df['pepname'] == 'A0A0G2JSH2_74']['score'].values[0]
    
    # 计算排名
    ranks = df['score'].rank(ascending=False).astype(int)
    rank_of_A0A0G2JSH2 = ranks[df[df['pepname'] == 'A0A0G2JSH2_74'].index[0]]
    
    # 将得分和排名加入到列表中
    scores_and_ranks.append((score, rank_of_A0A0G2JSH2))

# 打印结果
for count, (score, rank) in enumerate(scores_and_ranks, start=1):
    print(f"iDNN{count}中'A0A0G2JSH2'的得分为：{score}，排名为：{rank}")


iDNN1中'A0A0G2JSH2'的得分为：0.7017485，排名为：1
iDNN2中'A0A0G2JSH2'的得分为：0.563576，排名为：16
iDNN3中'A0A0G2JSH2'的得分为：0.44146675，排名为：56
iDNN4中'A0A0G2JSH2'的得分为：0.20108594，排名为：325
iDNN5中'A0A0G2JSH2'的得分为：0.22744238，排名为：284
